## Model: Mistral-7B-Instruct-v0.2-GPTQ
* Embedding - BAAI/bge-small-en-v1.5 Model
* Vectorizer - FAISS
* LLM Model - TheBloke/Mistral-7B-Instruct-v0.2-GPTQ Model
* HuggingFace Pipeline
* Retrieval and Context Augmentation

## Step1: Import Libraries

In [1]:
!pip install --upgrade --no-cache-dir \
  transformers \
  accelerate \
  sentence-transformers \
  langchain \
  datasets \
  bitsandbytes \
  langchain-community \
  auto-gptq --quiet

!pip install --upgrade --no-cache-dir "optimum[onnxruntime]" --quiet
!pip install faiss-cpu --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 126.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 111.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 222.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 95.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 260.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 212.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 184.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 272.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 185.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 209.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
from langchain_community.vectorstores import FAISS # type: ignore
from langchain.embeddings import HuggingFaceEmbeddings # type: ignore
from langchain.schema.document import Document # type: ignore
from sentence_transformers import SentenceTransformer # type: ignore
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline # type: ignore
from langchain_core.output_parsers import StrOutputParser # type: ignore
import torch # type: ignore
import logging
import pandas as pd

from google.colab import userdata # type: ignore
HF_TOKEN=userdata.get('HF_TOKEN')

In [ ]:
from google.colab import drive # type: ignore
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv("/content/drive/MyDrive/my_project/processed_cleaned_chunks.csv")
docs = [
    Document(
        page_content=row['text'],
        metadata={"chunk_id": row['chunk_id'], "video_id": row['video_id']}
    )
    for _, row in df.iterrows()
]
print(f"Loaded {len(docs)} documents for indexing/retrieval.")

Loaded 328 documents for indexing/retrieval.


## Step2: Building & Saving the Embeddings & FAISS Index

In [5]:
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

faiss_index = FAISS.from_documents(docs, embedding_model)
faiss_index.save_local("/content/drive/MyDrive/faiss_store")


/tmp/ipython-input-5-3515908250.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from huggingface_hub import login
login(token=HF_TOKEN)

## Step3: Load Model

In [ ]:
model_id = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    trust_remote_code=True
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:410: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:418: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:461: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float16)


model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
Some weights of the model checkpoint at TheBloke/Mistral-7B-Instruct-v0.2-GPTQ were not used when initializing MistralForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.layers.10.mlp.up_proj.bias', 'model.layers.10.self_attn.k_proj.bias', 'model.layers.10.self_attn.o_proj.bias', 'model.layers.10.self_attn.q_pr

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

## Step4: Loading & Using the Embedding & FAISS Index

In [ ]:
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")
faiss_index = FAISS.load_local("/content/drive/MyDrive/faiss_store", embedding_model, allow_dangerous_deserialization=True)
retriever = faiss_index.as_retriever(search_type="similarity", search_kwargs={"k": 5})

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Step5: HuggingFace Pipeline

In [9]:
from langchain.llms import HuggingFacePipeline
from transformers import pipeline
llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.1
)

llm = HuggingFacePipeline(pipeline=llm_pipeline)

Device set to use cuda:0
/tmp/ipython-input-9-283308427.py:14: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=llm_pipeline)


## Step6: Retrieval & Context Augmentation

In [ ]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

In [11]:

import logging
logging.getLogger("langchain_community.vectorstores").setLevel(logging.ERROR)
logging.getLogger("transformers").setLevel(logging.ERROR)
query = "How do I create a catalog item in ServiceNow?"
response = qa_chain(query)
raw_output = response["result"]
if "Answer:" in raw_output:
    cleaned = raw_output.split("Answer:")[-1].strip()
else:
    cleaned = raw_output.strip()
print(cleaned)

/tmp/ipython-input-11-4197205006.py:5: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain(query)


Creating a catalog item in ServiceNow involves several steps. First, you need to ensure that you have the necessary permissions to create catalog items. Next, you can navigate to the Catalog application in ServiceNow and select the type of catalog item you want to create (e.g., hardware, software, or service). Once you have selected the type, you will need to fill out various fields such as item details, pricing, and availability. After filling out all the required fields, you can submit the item for approval if required. Once approved, the item will be added to the catalog and can be ordered by users. For more detailed instructions, refer to the ServiceNow documentation or consult with your ServiceNow administrator.


## Step7: Test Queries

In [ ]:
logging.getLogger("langchain_community.vectorstores").setLevel(logging.ERROR)
logging.getLogger("transformers").setLevel(logging.ERROR)
query = "What is ITSM in ServiceNow?"
response = qa_chain(query)
raw_output = response["result"]

if "Answer:" in raw_output:
    cleaned = raw_output.split("Answer:")[-1].strip()
else:
    cleaned = raw_output.strip()
print(query)
print(cleaned)

What is ITSM in ServiceNow?
In the context provided, ITSM in ServiceNow refers to the IT Service Management solution offered by ServiceNow. This solution encompasses various aspects of IT service delivery and management, such as incident management, problem management, change management, and configuration management. The goal is to help organizations automate and streamline their IT operations, improve efficiency, and enhance the user experience. ServiceNow's ITSM offerings include IT Operations Management (ITOM) and other related modules that enable organizations to manage their IT services in a more effective and efficient manner. The AI-powered Service Operation feature discussed in the podcast enhances the capabilities of ITSM by enabling proactive problem detection and resolution, moving away from a reactive approach to incident management.


In [13]:
logging.getLogger("langchain_community.vectorstores").setLevel(logging.ERROR)
logging.getLogger("transformers").setLevel(logging.ERROR)
query = "Explain CMDB relationships."
response = qa_chain(query)
raw_output = response["result"]

if "Answer:" in raw_output:
    cleaned = raw_output.split("Answer:")[-1].strip()
else:
    cleaned = raw_output.strip()
print(query)
print(cleaned)

Explain CMDB relationships.
A Configuration Management Database (CMDB) is a database that holds configuration items (CIs) and their interrelationships. In the context of IT service management, a CI is anything that needs to be managed in order to deliver an IT service. The relationships between CIs represent dependencies and impacts between those managed elements. By understanding these relationships, IT teams can proactively identify potential issues and prevent incidents before they occur. The use of a knowledge graph is a modern and flexible approach to modeling these relationships, allowing data to be ingested in various formats and structures, enabling more accurate and comprehensive insights. For instance, in an ecommerce environment, alerts in ITOM can be cross-referenced with information from the CMDB, workflows, and previous incidents to suggest the best next steps for operators. Even without a fully matured CMDB, this approach can still yield valuable results by decoupling th

In [14]:
logging.getLogger("langchain_community.vectorstores").setLevel(logging.ERROR)
logging.getLogger("transformers").setLevel(logging.ERROR)
query = "How does Incident Management work?"
response = qa_chain(query)
raw_output = response["result"]

if "Answer:" in raw_output:
    cleaned = raw_output.split("Answer:")[-1].strip()
else:
    cleaned = raw_output.strip()
print(query)
print(cleaned)

How does Incident Management work?
In IT Service Management, Incident Management involves recognizing a change in target (trigger), which then kicks off a specific use case. This use case may involve multiple agents, each performing a distinct task. For instance, one agent might be responsible for finding portfolio cost information, another for identifying rationalization candidates, and a third for taking action and creating demand records. The goal is to quickly understand the impact and severity of the issue using various tools, allowing the Subject Matter Expert to begin investigating effectively. It's important to note that organizations often deal with numerous tools and systems while monitoring their applications and microservices, making it crucial to have a centralized solution like ITOM to manage these alerts and incidents efficiently. By moving from an incident creation culture to an alert-based actionable mindset, teams can be more proactive and resolve issues in a predicti

In [15]:
logging.getLogger("langchain_community.vectorstores").setLevel(logging.ERROR)
logging.getLogger("transformers").setLevel(logging.ERROR)
query = "What are different AI concepts used in ServiceNow?"
response = qa_chain(query)
raw_output = response["result"]
if "Answer:" in raw_output:
    cleaned = raw_output.split("Answer:")[-1].strip()
else:
    cleaned = raw_output.strip()
print(query)
print(cleaned)

What are different AI concepts used in ServiceNow?
ServiceNow uses AI across several aspects of its platform, including data integration, creating actionable insights using AI agents, and creating assisted and autonomous workflows. The AI agent is a significant component of this, as it operates 24/7, observes issues, receives information, and takes action on behalf of end users or teams, enabling businesses to meet their KPIs and objectives more efficiently. Additionally, ServiceNow integrates various large language models to build new use cases and manage them for customers in the future.


In [16]:
logging.getLogger("langchain_community.vectorstores").setLevel(logging.ERROR)
logging.getLogger("transformers").setLevel(logging.ERROR)
query = "What is CMDB in ServiceNow?"
response = qa_chain(query)
raw_output = response["result"]
if "Answer:" in raw_output:
    cleaned = raw_output.split("Answer:")[-1].strip()
else:
    cleaned = raw_output.strip()
print(query)
print(cleaned)

What is CMDB in ServiceNow?
The Configuration Management Database (CMDB) is a central repository for storing configuration data about IT infrastructure components in ServiceNow. It provides essential information for managing and maintaining IT systems and enables automated incident resolution through integrations with other ServiceNow applications such as ITSM, ITAM, and ITIL. Additionally, ServiceNow offers solutions like CMDb Amplified to help organizations improve their CMDB health and maturity.


## Metrics

In [32]:
new_results_df = pd.DataFrame(results)

OUTPUT_FILE = '/content/drive/MyDrive/model_outputs.csv'

if os.path.exists(OUTPUT_FILE):
    prev_results_df = pd.read_csv(OUTPUT_FILE)
    combined_df = pd.concat([prev_results_df, new_results_df], ignore_index=True)
else:
    combined_df = new_results_df
combined_df.to_csv(OUTPUT_FILE, index=False)
print(f"Appended results and saved to: {OUTPUT_FILE}")

Appended results and saved to: /content/drive/MyDrive/model_outputs.csv


In [33]:
questions = [
    "What is ITSM in ServiceNow?",
    "Explain CMDB relationships.",
    "How does Incident Management work?",
    "What are different AI concepts used in ServiceNow?",
    "What is CMDB in ServiceNow?"
]

reference_answers = [
    """ITSM (IT Service Management) in ServiceNow refers to the strategic approach for designing, delivering, managing, and improving the way IT is used within an organization. ServiceNow ITSM offers a cloud-based platform to automate and streamline core IT processes such as Incident Management, Problem Management, Change Management, Request Fulfillment, and Knowledge Management. It enhances service quality, reduces operational costs, and improves user satisfaction through intelligent workflows, analytics, and a centralized service portal. ITSM in ServiceNow aligns IT services with business goals, ensuring efficient and consistent service delivery.""",
    """CMDB (Configuration Management Database) relationships in ServiceNow define how Configuration Items (CIs) are connected and interact with each other. These relationships help build a dependency map that visually represents infrastructure, services, and their interconnections. Common relationship types include "Depends on / Used by," "Runs on / Hosted on," and "Connected to." For example, an application may run on a server, which in turn depends on a specific database. These relationships are essential for impact analysis, change planning, and root cause analysis, enabling IT teams to understand how changes or incidents affect related systems.""",
    """Incident Management in ServiceNow focuses on restoring normal service operations as quickly as possible after an unplanned interruption. When an incident is reported—via the service portal, email, or phone—it is automatically logged and categorized. The system assigns priority based on impact and urgency, then routes the ticket to the appropriate support group. Technicians investigate, resolve the issue, and document the resolution. Users are kept informed through notifications. Once resolved, the incident is closed after verification. Incident Management helps minimize downtime, improves user satisfaction, and provides valuable data for trend analysis and continual service improvement.""",
    """ServiceNow leverages several AI concepts to enhance automation and user experience. Key AI features include Natural Language Understanding (NLU) for interpreting user queries, Predictive Intelligence for categorizing and routing incidents using machine learning, and Virtual Agents that provide instant, conversational support. It also uses AI Search to deliver accurate, context-aware search results and Recommendation Engines to suggest knowledge articles or solutions. Additionally, Process Optimization uses AI to analyze workflows and identify bottlenecks. These AI tools improve service efficiency, reduce manual work, and enable proactive IT support across the platform.""",
    """CMDB (Configuration Management Database) in ServiceNow is a centralized repository that stores information about Configuration Items (CIs)—such as servers, applications, databases, and network devices—and their relationships. It helps organizations track the state and dependencies of IT assets across their lifecycle. CMDB enables accurate impact analysis, change management, and incident resolution by providing a complete view of the IT environment. In ServiceNow, CMDB is tightly integrated with ITSM processes, helping ensure data accuracy through automated discovery, reconciliation, and ongoing updates, thereby supporting better decision-making and service delivery."""
]

In [34]:
model_answers = [
    "In the context provided, ITSM in ServiceNow refers to the IT Service Management solution offered by ServiceNow. This solution encompasses various aspects of IT service delivery and management, such as incident management, problem management, change management, and configuration management. The goal is to help organizations automate and streamline their IT operations, improve efficiency, and enhance the user experience. ServiceNow's ITSM offerings include IT Operations Management (ITOM) and other related modules that enable organizations to manage their IT services in a more effective and efficient manner. The AI-powered Service Operation feature discussed in the podcast enhances the capabilities of ITSM by enabling proactive problem detection and resolution, moving away from a reactive approach to incident management.",
    "A Configuration Management Database (CMDB) is a database that holds configuration items (CIs) and their interrelationships. In the context of IT service management, a CI is anything that needs to be managed in order to deliver an IT service. The relationships between CIs represent dependencies and impacts between those managed elements. By understanding these relationships, IT teams can proactively identify potential issues and prevent incidents before they occur. The use of a knowledge graph is a modern and flexible approach to modeling these relationships, allowing data to be ingested in various formats and structures, enabling more accurate and comprehensive insights. For instance, in an ecommerce environment, alerts in ITOM can be cross-referenced with information from the CMDB, workflows, and previous incidents to suggest the best next steps for operators. Even without a fully matured CMDB, this approach can still yield valuable results by decoupling the dependency on having a perfect CMDB. Additionally, there are specialized agents that can assist in tasks such as identifying relevant knowledge base articles, summarizing their contents, and suggesting the most appropriate one for a given issue. These agents can also draw from external sources of industry knowledge and observability tools to provide more complete insights.",
    "In IT Service Management, Incident Management involves recognizing a change in target (trigger), which then kicks off a specific use case. This use case may involve multiple agents, each performing a distinct task. For instance, one agent might be responsible for finding portfolio cost information, another for identifying rationalization candidates, and a third for taking action and creating demand records. The goal is to quickly understand the impact and severity of the issue using various tools, allowing the Subject Matter Expert to begin investigating effectively. It's important to note that organizations often deal with numerous tools and systems while monitoring their applications and microservices, making it crucial to have a centralized solution like ITOM to manage these alerts and incidents efficiently. By moving from an incident creation culture to an alert-based actionable mindset, teams can be more proactive and resolve issues in a predictive manner.",
    "ServiceNow uses AI across several aspects of its platform, including data integration, creating actionable insights using AI agents, and creating assisted and autonomous workflows. The AI agent is a significant component of this, as it operates 24/7, observes issues, receives information, and takes action on behalf of end users or teams, enabling businesses to meet their KPIs and objectives more efficiently. Additionally, ServiceNow integrates various large language models to build new use cases and manage them for customers in the future.",
    "The Configuration Management Database (CMDB) is a central repository for storing configuration data about IT infrastructure components in ServiceNow. It provides essential information for managing and maintaining IT systems and enables automated incident resolution through integrations with other ServiceNow applications such as ITSM, ITAM, and ITIL. Additionally, ServiceNow offers solutions like CMDb Amplified to help organizations improve their CMDB health and maturity."
    ]

In [35]:
from datetime import datetime
def exact_match(prediction, reference):
    return int(prediction.strip().lower() == reference.strip().lower())

def f1_score(prediction, reference):
    pred_tokens = prediction.lower().split()
    ref_tokens = reference.lower().split()
    common = set(pred_tokens) & set(ref_tokens)
    if not common:
        return 0.0
    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(ref_tokens)
    if precision + recall == 0:
        return 0.0
    return 2 * (precision * recall) / (precision + recall)

def rouge_l_score(prediction, reference):
    def lcs(X, Y):
        m, n = len(X), len(Y)
        dp = [[0] * (n + 1) for _ in range(m + 1)]
        for i in range(m):
            for j in range(n):
                if X[i] == Y[j]:
                    dp[i + 1][j + 1] = dp[i][j] + 1
                else:
                    dp[i + 1][j + 1] = max(dp[i][j + 1], dp[i + 1][j])
        return dp[m][n]
    pred_tokens = prediction.lower().split()
    ref_tokens = reference.lower().split()
    lcs_len = lcs(pred_tokens, ref_tokens)
    if len(ref_tokens) == 0 or len(pred_tokens) == 0:
        return 0.0
    recall = lcs_len / len(ref_tokens)
    precision = lcs_len / len(pred_tokens)
    if recall + precision == 0:
        return 0.0
    return 2 * recall * precision / (recall + precision)

def bleu_score(prediction, reference):
    pred_tokens = prediction.lower().split()
    ref_tokens = reference.lower().split()
    if not pred_tokens or not ref_tokens:
        return 0.0
    matches = sum(1 for token in pred_tokens if token in ref_tokens)
    return matches / len(pred_tokens)

In [ ]:
results = []
for question, ref_answer in zip(questions, reference_answers):
    response = qa_chain(question)
    model_answer = response["result"]
    if "Answer:" in model_answer:
        model_answer = model_answer.split("Answer:")[-1].strip()
    else:
        model_answer = model_answer.strip()
    timestamp = datetime.now().isoformat()
    em = exact_match(model_answer, ref_answer)
    f1 = f1_score(model_answer, ref_answer)
    rouge = rouge_l_score(model_answer, ref_answer)
    bleu = bleu_score(model_answer, ref_answer)
    print(f"Q: {question}")
    print(f"Model: {model_answer}")
    print(f"Reference: {ref_answer[:100]}...")  # Truncate for display
    print(f"EM: {em}, F1: {f1:.3f}, ROUGE-L: {rouge:.3f}, BLEU: {bleu:.3f}")
    print("-" * 60)
    results.append({
        "timestamp": timestamp,
        "notebook":'3b_model_test_mistral7B',
        "query": question,
        "model_answer": model_answer,
        "em": em,
        "f1": f1,
        "rougeL": rouge,
        "bleu": bleu
    })

results_df = pd.DataFrame(results)
results_df.to_csv(OUTPUT_FILE, index=False)
print(f"\nSaved results to: {OUTPUT_FILE}")

Q: What is ITSM in ServiceNow?
Model: ITSM stands for IT Service Management in ServiceNow. It refers to the practices, processes, and tools used by organizations to effectively deliver and manage technology services to support their business operations. In the context of ServiceNow, ITSM includes modules such as Incident Management, Problem Management, Change Management, and Service Catalog to help organizations automate and improve their IT service delivery and support functions.
Reference: ITSM (IT Service Management) in ServiceNow refers to the strategic approach for designing, deliverin...
EM: 0, F1: 0.255, ROUGE-L: 0.309, BLEU: 0.556
------------------------------------------------------------
Q: Explain CMDB relationships.
Model: A Configuration Management Database (CMDB) is a system of record for managing configurations in an IT infrastructure. In the context of ServiceNow, the CMDB can be extended using advanced data processing techniques such as Data Fabric and RAPID-RB to han